In [1]:
import pandas as pd
import numpy as np

In [2]:
#       - Importación -

df = pd.read_csv('/content/nyc-rolling-sales-cured.csv', dtype=str)
df = df.replace('-', '', regex=True)
df.columns = df.columns.str.replace(' ', '_')
df.shape

<ipython-input-2-2f7258474ea9>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('-', '', regex=True)


(84548, 22)

In [3]:
#       - Formato -

convert_types = {
    'Unnamed:_0': 'str',
    'BOROUGH': 'int',
    'NEIGHBORHOOD': 'str',
    'BUILDING_CLASS_CATEGORY': 'str',
    'TAX_CLASS_AT_PRESENT': 'category',
    'BLOCK': 'int',
    'LOT': 'int',
    'EASE-MENT': 'str',
    'BUILDING_CLASS_AT_PRESENT': 'category',
    'ADDRESS': 'str',
    'APARTMENT_NUMBER': 'str',
    'ZIP_CODE': 'int',
    'RESIDENTIAL_UNITS': 'int',
    'COMMERCIAL_UNITS': 'int',
    'TOTAL_UNITS': 'int',
    'LAND_SQUARE_FEET': 'int',
    'GROSS_SQUARE_FEET': 'int',
    'YEAR_BUILT': 'int',
    'TAX_CLASS_AT_TIME_OF_SALE': 'category',
    'BUILDING_CLASS_AT_TIME_OF_SALE': 'category',
    'SALE_PRICE': 'float',
    'SALE_DATE': 'datetime64[ns]'
}

for col, dtype in convert_types.items():
    if col not in df.columns:
        continue
    if dtype == 'datetime64[ns]':
        df[col] = pd.to_datetime(df[col], errors='coerce')
    elif dtype in ['int', 'float']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else:
        df[col] = df[col].astype(dtype)


print(df.columns)


Index(['Unnamed:_0', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')


In [4]:
#       - Filtro -
df['SALE_PRICE'] = pd.to_numeric(df['SALE_PRICE'], errors='coerce')
df.shape

(84548, 22)

In [5]:
#       - Limpieza -
df = df[df['SALE_PRICE'] > 10000]
df = df[~((df['RESIDENTIAL_UNITS'] > 3))]
df = df[~((df['COMMERCIAL_UNITS'] > 3))]
df_ready = df.drop(['EASE-MENT','Unnamed:_0','APARTMENT_NUMBER'],axis='columns')

df_ready.shape

(56008, 19)

In [6]:
#       - Depuración de GROSS_SQUARE_FEET -

df_ready['GROSS_SQUARE_FEET'] = df_ready['GROSS_SQUARE_FEET'].astype(str).str.replace(',', '').str.strip()
df_ready['GROSS_SQUARE_FEET'] = df_ready['GROSS_SQUARE_FEET'].replace(['', '-', '0'], np.nan)
df_ready['GROSS_SQUARE_FEET'] = pd.to_numeric(df_ready['GROSS_SQUARE_FEET'], errors='coerce')
df_ready['GROSS_SQUARE_FEET'] = df_ready['GROSS_SQUARE_FEET'].replace(0, np.nan)

#       - Imputación de GROSS_SQUARE_FEET-

missing_data = df_ready['GROSS_SQUARE_FEET'].isna()
imputation = df_ready.groupby('BUILDING_CLASS_CATEGORY')['GROSS_SQUARE_FEET'].transform('median')
df_ready.loc[missing_data, 'GROSS_SQUARE_FEET'] = df_ready.loc[missing_data, 'GROSS_SQUARE_FEET'].fillna(imputation)
median_global = df_ready['GROSS_SQUARE_FEET'].median()

df_ready.loc[missing_data, 'GROSS_SQUARE_FEET'] = median_global

In [7]:
#       - Filtro "GROSS_SQUARE_FEET" final-

df_ready = df_ready[df_ready['GROSS_SQUARE_FEET'] > 0]
df_ready.shape

(56008, 19)

In [8]:
#       - Exportación -
df_ready.to_csv('resultado_NYC.csv', index=False, encoding='utf-8', float_format='%.2f')